## Preparación de los dataframes para la base de datos

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Agregar el directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar la función suma del módulo auxiliares
import funciones_auxiliares as fa

In [2]:
df = pd.read_csv(f"../datos/df_supermercados.csv")

Tabla supermercados

In [3]:
df_super = pd.DataFrame(df["supermercado"].unique())
df_super.reset_index(inplace=True)
df_super.rename(columns={'index':'id_super', 0:'nombre'}, inplace=True)
df_super["id_super"]=df_super["id_super"].apply(lambda x: x+1)

In [4]:
df_super.to_csv("../datos/tabla_super.csv")

In [5]:
df_super

,id_super,nombre
0,1,alcampo
1,2,carrefour
2,3,dia
3,4,eroski
4,5,hipercor
5,6,mercadona


Tabla productos

In [6]:
df_productos = pd.DataFrame(df["producto"].unique())
df_productos.reset_index(inplace=True)
df_productos.rename(columns={'index':'id_producto', 0:'nombre'}, inplace=True)
df_productos["id_producto"]=df_productos["id_producto"].apply(lambda x: x+1)

In [7]:
df_productos.to_csv("../datos/tabla_productos.csv")

In [8]:
df_productos

,id_producto,nombre
0,1,aceite de girasol
1,2,aceite de oliva
2,3,leche


Tabla comparativa

Incluimos los id del supermercado y producto para tener las claves foraneas:

In [9]:
df_merged = df.merge(right=df_super, how='left', left_on ='supermercado', right_on="nombre")
df_merged=df_merged.drop(columns=["Unnamed: 0", "supermercado", "nombre"])

In [10]:
df_merged_2 = df_merged.merge(right=df_productos, how='left', left_on ='producto', right_on="nombre")
df_merged_2 = df_merged_2.drop(columns=["producto", "nombre"])

Transformamos el precio a float:

In [11]:
df_merged_2["Precio (€)"]=df_merged_2["Precio (€)"].str.replace(",",".").apply(float)
df_merged_2["porcentaje"]=df_merged_2["porcentaje"].str.replace(",",".").apply(float)

Transformamos la fecha a formato fecha:

In [12]:
df_merged_2["Día"]=pd.to_datetime(df_merged_2["Día"])

Reordenamos y borramos duplicados:

In [13]:
df_merged_2 = df_merged_2.reindex(columns=['id_super', 'id_producto', 'nombre_producto', 'subcategoria', 'Día', 'Precio (€)', 'incremento', 'porcentaje'])
df_merged_2.drop_duplicates(inplace=True)

Finalmente añadiremos tres nuevas columnas: "marca", "cantidad" y "dia_semana".

In [23]:
# Primero corregimos algunos nombres mal escritos:
df_merged_2["nombre_producto"]=df_merged_2["nombre_producto"].str.replace("carbonel", "carbonell")
df_merged_2["nombre_producto"]=df_merged_2["nombre_producto"].str.replace("arginano", "arguinano")

In [24]:
# Creamos el df con las nuevas columnas
df_aux = pd.DataFrame()
df_aux[['marca', 'cantidad_unidad']] = df_merged_2['nombre_producto'].apply(fa.extraer_marca_cantidad)
df_aux["marca"]=df_aux["marca"].str.strip()

In [25]:
df_merged_3 = pd.concat([df_merged_2, df_aux], axis=1)

In [31]:
# Añadimos el día de la semana basandonos en la fecha
dias_traduccion = {
    'Monday': 'lunes', 'Tuesday': 'martes', 'Wednesday': 'miércoles', 
    'Thursday': 'jueves', 'Friday': 'viernes', 'Saturday': 'sábado', 'Sunday': 'domingo'
}
df_merged_3['dia_semana']=df_merged_3['Día'].dt.day_name().map(dias_traduccion)

In [33]:
df_merged_3.to_csv("../datos/tabla_comparativa.csv")

In [34]:
df_merged_3.sample(5)

,id_super,id_producto,nombre_producto,subcategoria,Día,Precio (€),incremento,porcentaje,marca,cantidad_unidad,dia_semana
75912,4,3,leche desnatada calcio pascual brik 1 litro,desnatada,2024-08-28,1.75,NaN,NaN,pascual,1 l,miércoles
65917,4,2,aceite de oliva 04 olilan botella 1 litro,NaN,2024-07-04,7.62,NaN,NaN,olilan,1 l,jueves
1272,1,1,producto alcampo aceite de girasol botella de 1 l,NaN,2024-07-09,1.45,NaN,NaN,alcampo,1 l,martes
64394,4,1,aceite de girasol lanisol botella 1 litro,NaN,2024-07-22,1.45,NaN,NaN,lanisol,1 l,lunes
15926,1,3,auchan leche desnatada de vaca sin lactosa 6 x...,desnatada,2024-09-12,5.10,NaN,NaN,auchan,6 x 1l,jueves
